In [13]:
from scripts.tools import active_dev_worksheet as adw
import networkx as nx
from pywintypes import com_error
import collections
import gc
import time

In [14]:
def graph_all(issues):
    g = nx.DiGraph()
    for i in issues:
        iid = i.id
        g.add_node(iid)
        pid = i.parent
        if pid is not None:
            g.add_edge(pid, iid)
    if not nx.is_forest(g):  # should not be possible
        raise ValueError("Circles in graph :(") 
    return g
     
    
def _get_versioned_roots(g, idmap):
    allroots = find_roots(g)
    fvroots = collections.defaultdict(list)
    for root in allroots:
        iss = idmap[root]
        fvn = adw.api.resource_attrib(iss.fixed_version, "name")
        fvroots[fvn].append(root)
    return fvroots
    
def find_roots(g):
    return [n for n, ind in g.in_degree() if ind == 0]
    
def _dfs_visit(g, parent, visit, depth):
    for node in sorted(g.successors(parent)):
        visit(parent, node, depth)
        _dfs_visit(g, node, visit, depth + 1)

        
def dfs_visit(g, node, visit):
    visit(None, node, 0)
    _dfs_visit(g, node, visit, 1)

def _gv_visitor(graph):
    def visit(parent, node, _depth):
        graph.add_node(node)
        if parent is not None:
            graph.add_edge(parent, node)
    return visit    
    
def graph_versions(allg, idmap, versions):
    
    fvroots = _get_versioned_roots(allg, idmap)
    graphs = {}
    for fv in versions:
        graphs[fv.name] = subg = nx.DiGraph()
        roots = fvroots.get(fv.name, ())
        for r in roots:
            dfs_visit(allg, r, _gv_visitor(subg))
            
    # for un-sprinted issues, we only care about
    # ones for PBS Software
    sw_proj_id = 5
    no_sprint = fvroots.get(None, ())
    graphs['<No Sprint>'] = subg = nx.DiGraph()
    for r in no_sprint:
        iss = idmap[r]
        if iss.project.id == sw_proj_id:
            dfs_visit(allg, r, _gv_visitor(subg))
        
    return graphs

In [20]:
def download(key, proj_identifier):
    url = "https://issue.pbsbiotech.com"
    client = adw.api.Client(url, key)
    print("downloading issues...")
    issues = client.Issues.filter(status_id="*")
    idmap = {i.id:i for i in issues}
    print("downloading versions...")
    versions = client.Projects.filter_versions("pbssoftware")
    return issues, idmap, versions

In [35]:
def make_sheets(wb, vgraphs, idmap):
    print("preparing to make worksheets...")
    ws = wb.Worksheets.Add()
    ws.UsedRange.Clear()
    with adw.screen_lock(wb.Application):
        tc = adw.TableCreator()
        for i, (name, vg) in enumerate(vgraphs.items()):
            if name is None:
                name = "<Uncategorized>"
            print(f"creating worksheet '{name}'")
            if i != 0:
                ws = wb.Worksheets.Add()
            ws.Name = name
            tc.create(ws, vg, idmap)
        wb.Worksheets("Sheet1").Delete()

def _filter_versions(versions):
    return [v for v in versions if v.status == "open"]
    
def init_excel():
    print("Opening excel...")
    xl = adw.open_excel()
    wb = xl.Workbooks.Add()
    return xl, wb

def graphify(key, proj_id):
    issues, idmap, versions = download(key, proj_id)
    versions = _filter_versions(versions)
    allg = graph_all(issues)
    graphs = graph_versions(allg, idmap, versions)
    return graphs, idmap

def save_to_sharepoint(wb, fp):
    print("Saving to sharepoint...")
    try:
        adw.save_to_sharepoint(wb, save_path)
    except com_error:
        print("Save failed :(")
        raise
    else:
        print("Success! Wrapping up...")
        
def main(key, path):
    vgraphs, idmap = graphify(key, "pbssoftware")
    xl, wb = init_excel()
    make_sheets(wb, vgraphs, idmap)
    wb.Worksheets("Active Development").Activate()
    try:
        save_to_sharepoint(wb, path)
    finally:
        gc.collect()

In [36]:
save_path = "https://pbsbiotech.sharepoint.com/sites/SoftwareEngineeringLV1" + \
        "/Shared Documents/Project Management/Software Active Development.xlsx"
key = "7676add9cac6631410403671cdd7850311987898"
main(key, save_path)

downloading issues...
downloading versions...
Opening excel...
preparing to make worksheets...
creating worksheet 'Future Release'
creating worksheet '3.0'
creating worksheet 'Legacy'
creating worksheet '3.1.0'
creating worksheet 'IM229 C'
creating worksheet 'IC03405 J'
creating worksheet 'Active Development'
creating worksheet '<No Sprint>'
Saving to sharepoint...
Success! Wrapping up...
